In [1]:
#1
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
%matplotlib inline
plt.style.use('seaborn-whitegrid')
plt.style.use('fivethirtyeight')
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')


data_customer=pd.read_csv('/content/drive/MyDrive/Mockup Datasets/customer.csv')
data_product=pd.read_csv('/content/drive/MyDrive/Mockup Datasets/product.csv', error_bad_lines=False)
data_transactions=pd.read_csv('/content/drive/MyDrive/Mockup Datasets/transactions.csv')
data_click_stream=pd.read_csv('/content/drive/MyDrive/Mockup Datasets/click_stream.csv')

customer_clean=data_customer.dropna()
click_clean=data_click_stream.dropna()
product_clean=data_product.dropna()
transaction_clean=data_transactions.dropna()

#1. Merge data customer dengan transaction menggunakan customer_id
customer_and_transactions = customer_clean.merge(transaction_clean, on='customer_id', validate='m:m')

#2. Merge data customer_and_transactions dengan data click stream menggunakan session_id
customer_transactions_click = customer_and_transactions.merge(click_clean, on='session_id', validate='m:m')

#2.1. Mengambil data product id dari kolom product_metadata
customer_transactions_click['product_id']=customer_transactions_click.product_metadata.str[15:21]


data1_clean=customer_transactions_click.dropna()
product_clean.rename(columns = {'id':'product_id'}, inplace = True)

#3. Merging data1 dengan data product melalui product_id
data1_clean['product_id'] = data1_clean['product_id'].astype(str)
product_clean['product_id'] = product_clean['product_id'].astype(str)
customer_transactions_click_product = data1_clean.merge(product_clean, on='product_id', how='outer')

data_final=customer_transactions_click_product

# CREATE VARIABLE IS_CHURN
data_final = data_final.dropna(subset=['created_at'])

data_final['created_at'] = data_final['created_at'].str.replace("T"," ")
data_final['created_at'] = data_final['created_at'].str.replace("Z","")

data_final['created_at']=pd.to_datetime(data_final['created_at'])

data_final2=data_final.drop_duplicates(subset=['customer_id'], keep='last').sort_values(by='customer_id')

import datetime as dt
present_day = data_final2['created_at'].max() 
data_final2['recency']= data_final2['created_at'].apply(lambda x: (present_day - x).days)

data_final2['is_churn'] = np.where((data_final2['recency'] >= 30) , 1, 0)

churn_customer = data_final2.drop_duplicates(subset='created_at').reset_index()

#CREATE INDEPENDENT VARIABLE
#Data usia
data_final2['birthdate'] = pd.to_datetime(data_final2.birthdate)
from datetime import datetime
data_final2['Today'] = datetime.now().date()
data_final2['Today'] = pd.to_datetime(data_final2.Today)
data_final2['Age']= data_final2.Today - data_final2.birthdate
data_final2['new']=data_final2.Age.dt.days
data_final2['Age_inYear']=data_final2.new/365
data_final2['Age_inYear2']=data_final2['Age_inYear'].astype(int)
data_final2['age'] = data_final2['Age_inYear2']

#Data jenis kelamin
data_final2['isFemale'] = np.where(
      (data_final2['gender_x']== 'F') , 1, 0
    )

#Data tipe perangkat
data_final2['is_iOS'] = np.where(
      (data_final2['device_type']== 'iOS') , 1, 0
    )




#2
dff1=data_final2
dff=dff1.drop(['gender_y', 'masterCategory','subCategory','articleType','baseColour','season',
         'year','usage','productDisplayName','Age_inYear','Age','age','new'], axis=1)

#dft['created_at] = pd.to_datetime(dft['created_at],utc=True)
dff['created_at'] = pd.to_datetime(dff['created_at']).dt.tz_localize(None)

#dft['created_hour']=dft['created_at'].dt.hour
dff['created_day']=dff['created_at'].dt.day_name()
dff['created_month_year'] = dff['created_at'].apply(lambda x: x.strftime('%Y-%m')) 
dff['created_year']=dff['created_at'].dt.year
dff['created_month']=dff['created_at'].dt.month_name()

dff['created_time'] = (dff['created_at'].dt.hour % 24 + 4) // 4
dff['created_time'].replace({1: 'Late Night',
                      2: 'Early Morning',
                      3: 'Morning',
                      4: 'Noon',
                      5: 'Evening',
                      6: 'Night'}, inplace=True)
mod = dff.copy()

replacements = {'payment_status': {'Success': 1, 'Failed': 0}, 
                'traffic_source': {'WEB': 0, 'MOBILE': 1}}

mod.replace(replacements, inplace=True)

# get the dummies and store it in a variable
dummies = pd.get_dummies(mod[['created_time','created_day','event_name','promo_code','payment_method']])
 
# Concatenate the dummies to original dataframe
mod2 = pd.concat([mod, dummies], axis='columns')
 
# drop the values
mod2_1=mod2.drop(['created_time','created_day','event_name','promo_code','payment_method'], axis='columns')

mod3=mod2_1[['home_location_lat','home_location_long','promo_amount','shipment_fee',
         'shipment_location_lat','shipment_location_long','total_amount','recency',
         'is_churn','Age_inYear2','isFemale','is_iOS','created_time_Early Morning', 
         'created_time_Evening','created_time_Late Night', 'created_time_Morning', 
         'created_time_Night','created_time_Noon', 'created_day_Friday', 'created_day_Monday',
         'created_day_Saturday', 'created_day_Sunday', 'created_day_Thursday',
         'created_day_Tuesday', 'created_day_Wednesday','event_name_ADD_TO_CART', 
         'event_name_BOOKING', 'event_name_SEARCH','promo_code_AZ2022', 'promo_code_BUYMORE', 
         'promo_code_LIBURDONG','promo_code_SC2022', 'promo_code_STARTUP', 'promo_code_WEEKENDMANTAP',
         'promo_code_WEEKENDSERU', 'promo_code_XX2022','payment_method_Credit Card', 
         'payment_method_Debit Card','payment_method_Gopay', 'payment_method_LinkAja', 
         'payment_method_OVO']]

#3
#Keeping necessary column
data=mod3

#Choose variable
data_xy2=data[['promo_amount', 'shipment_fee', 'total_amount', 'Age_inYear2', 'isFemale', 'is_iOS', 'payment_method_OVO', 
               'created_time_Noon', 'created_day_Tuesday', 'event_name_ADD_TO_CART', 'promo_code_AZ2022', 'is_churn']]

y= data_xy2['is_churn']
X= data_xy2.drop(['is_churn'],1)

MessageError: ignored

In [ ]:
#XGB TUNING HYPERPARAMETER
#evaluation
from sklearn import metrics
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.metrics import make_scorer,accuracy_score,roc_auc_score,precision_score,recall_score,f1_score,log_loss
from sklearn.metrics import confusion_matrix
from numpy import loadtxt
from xgboost import XGBClassifier

# import packages for hyperparameters tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

model = XGBClassifier()
space={'max_depth': hp.choice("max_depth", np.arange(3, 18, 1, dtype=int)),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 0
    }

#Objective function
import xgboost as xgb
def objective(space):
    clf=xgb.XGBClassifier(
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']))
    
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    clf.fit(X_train, y_train,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=10,verbose=False)
    

    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred>0.5)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }

#Optimize algorithm
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.model_selection import StratifiedKFold 

xgb_classifier = XGBClassifier()
xgbc = StratifiedKFold(n_splits=2)

xgbc = GridSearchCV(estimator=xgb_classifier, 
                 param_grid={
                     'max_depth': [hp.quniform("max_depth", 3, 18, 1)],
                     'gamma': [hp.uniform ('gamma', 1,9)],
                     'reg_alpha' : [hp.quniform('reg_alpha', 40,180,1)],
                     'reg_lambda' : [hp.uniform('reg_lambda', 0,1)],
                     'colsample_bytree' : [hp.uniform('colsample_bytree', 0.5,1)],
                     'min_child_weight' : [hp.quniform('min_child_weight', 0, 10, 1)],
                     'n_estimators': [180],
                     'seed': [0]
                      },
                 
                 cv = StratifiedKFold(n_splits=2), 
                 verbose = 4)

tuning_model = XGBClassifier(**best_hyperparams)

#Training model
model_fit = tuning_model.fit(X_train,y_train)

#model performance using train dataset
y_pred_train=model_fit.predict(X_train)
print(classification_report(y_train,y_pred_train))

#model performance using test dataset
y_pred_test=model_fit.predict(X_test)
print(classification_report(y_test,y_pred_test))

In [ ]:
from sklearn.preprocessing import StandardScaler
import pickle

sc = StandardScaler()
X_scaled = sc.fit_transform(X)

filename = 'data_scaling_team_italy.pkl' 
pickle.dump(X_scaled, open(filename, 'wb'))